# Import Libary

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import RandomizedSearchCV


# Download Dataset pada Google Drive

In [ ]:
 !gdown 1Ddsq6ZfM5qSa7NISqpChxTx--Ckk5ZXF

Downloading...
From: https://drive.google.com/uc?id=1Ddsq6ZfM5qSa7NISqpChxTx--Ckk5ZXF
To: /content/Dataset_Giziwise.csv
100% 104k/104k [00:00<00:00, 3.88MB/s]


In [ ]:
# Load the dataset
file_path = '/content/Dataset_Giziwise.csv'
dataset = pd.read_csv(file_path, delimiter=';')



# Tampilkan Kolom dataset



In [ ]:
dataset.columns

Index(['Kode', 'Nama_bahan', 'Sumber', 'Air', 'ENERGI', 'PROTEIN', 'LEMAK',
       'KH', 'SERAT', 'ABU', 'KALSIUM', 'FOSFOR', 'BESI', 'NATRIUM', 'KALIUM',
       'TEMBAGA', 'SENG', 'RETINOL', 'B-KAR', 'KAR -TOTAL', 'THIAMIN',
       'RIBOFLAVIN', 'NIASIN', 'VIT_C', 'BDD'],
      dtype='object')

# Menampilkan sampel Data pada dataset

In [ ]:
dataset.sample(10)

,Kode,Nama_bahan,Sumber,Air,ENERGI,PROTEIN,LEMAK,KH,SERAT,ABU,...,TEMBAGA,SENG,RETINOL,B-KAR,KAR -TOTAL,THIAMIN,RIBOFLAVIN,NIASIN,VIT_C,BDD
754,GR063,ikan tarmon segar,KZGMI-2001,76.7,96,19.5,1.1,2.0,0.0,0.8,...,0.1,0.5,0,0,0,0.15,0.02,1.7,0,0.0
683,FP080,soto padang masakan,KZGPI-1990,75.0,127,5.9,6.6,11.0,0.7,2.0,...,0.0,0,12,0,204,0.22,0.00,0,0,100.0
921,KP002,margarin,DABM-1964,15.5,720,0.6,81,0.4,0.0,2.5,...,0.0,0,606,633,0,0,0.00,0,0,100.0
735,GR044,ikan malalugis segar,KZGMI-2001,75.6,99,20.6,1.4,0.9,0.0,1.5,...,0.2,0.9,661,0,0,0.3,0.02,0.5,0,0.0
115,AP095,padamaran,KZGMI-2001,676.0,171,39.0,89,188.0,0.0,8.0,...,0.0,0,0,0,0,0,0.00,0,0,100.0
269,CR026,kacang merah kering,KZGPI- l990,177.0,314,221.0,11,562.0,4.0,29.0,...,65.0,26,0,129,0,4,72.00,22,0,100.0
887,HP006,bayau mi balu\nmasakan,KGZMI-2001,36.1,313,6.0,12.2,44.8,0.0,0.9,...,0.0,0,783,89,0,0.14,0.27,9.7,0,100.0
181,BP014,bagea kelapa manis,KZGMI-2001,3.0,452,138.0,806,0.0,13.0,38.0,...,2.0,0,0,0,0,0,0.00,0,0,100.0
734,GR043,ikan lidah segar,KZGPI- 1990,78.3,104,15.6,3.2,2.0,0.0,0.9,...,0.0,0,33,0,0,0.03,0.00,0,0,0.0
471,ER029,gatep segar,KZGPI- 1990,52.7,191,4.5,1.5,39.7,13.4,1.6,...,0.7,1.3,0,51,0,0.05,0.20,1.8,0,100.0


# Prosesing

In [ ]:
nama_makanan = input("Masukkan nama makanan: ")
searched_data =  dataset[dataset["Nama_bahan"] == nama_makanan]
searched_data[['Nama_bahan','ENERGI','PROTEIN','LEMAK']]

Masukkan nama makanan: ikan lidah segar


,Nama_bahan,ENERGI,PROTEIN,LEMAK
734,ikan lidah segar,104,15.6,3.2


In [ ]:
# Converting columns to numeric, coercing errors to NaN
columns_to_convert = ['Air', 'PROTEIN', 'LEMAK', 'KH', 'SERAT', 'ABU', 'KALSIUM', 'FOSFOR',
                      'BESI', 'NATRIUM', 'KALIUM', 'TEMBAGA', 'SENG', 'RETINOL', 'B-KAR',
                      'KAR -TOTAL', 'THIAMIN', 'RIBOFLAVIN', 'NIASIN', 'VIT_C', 'BDD']
dataset[columns_to_convert] = dataset[columns_to_convert].apply(pd.to_numeric, errors='coerce')

In [ ]:
# Dropping rows with any NaN values after conversion
dataset_cleaned = dataset.dropna()

In [ ]:
# Feature Engineering: Polynomial Features
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
features_poly = poly.fit_transform(dataset_cleaned[columns_to_convert])
features = pd.DataFrame(features_poly, columns=poly.get_feature_names_out(columns_to_convert))

target = dataset_cleaned['ENERGI']

In [ ]:
# Scaling the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)

In [ ]:
# Scaling the target
target_scaler = MinMaxScaler()
target_scaled = target_scaler.fit_transform(target.values.reshape(-1, 1))

In [ ]:
# Memisahkan data menjadi set pelatihan dan pengujian
X_train, X_test, y_train, y_test = train_test_split(X_scaled, target_scaled, test_size=0.2, random_state=42)

In [ ]:
# Hyperparameter grid
param_dist_rf = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

In [ ]:
# Randomized search
random_search_rf = RandomizedSearchCV(estimator=RandomForestRegressor(random_state=42),
                                      param_distributions=param_dist_rf,
                                      n_iter=100, cv=5, verbose=2, random_state=42, n_jobs=-1)

random_search_rf.fit(X_train, y_train.ravel())
best_rf_model = random_search_rf.best_estimator_
y_pred_rf = best_rf_model.predict(X_test)
rf_accuracy = r2_score(y_test, y_pred_rf)

print("Random Forest Tuned Accuracy with Randomized Search:", rf_accuracy)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Random Forest Tuned Accuracy with Randomized Search: 0.8877593928158951


# Prediksi Nilai Nutrisi

In [ ]:
def predictor(nama_makanan: str, dataset: pd.DataFrame, scaler, model, target_scaler, poly, portion_size: float) -> pd.DataFrame:
    searched_data = dataset[dataset["Nama_bahan"] == nama_makanan]
    if searched_data.empty:
        raise ValueError(f"No data found for the food item: {nama_makanan}")

    numeric_value = searched_data[columns_to_convert].to_numpy()
    numeric_value_df = pd.DataFrame(numeric_value, columns=columns_to_convert)  # Buat DataFrame dengan nama kolom yang benar
    numeric_value_poly = poly.transform(numeric_value_df)
    numeric_value_poly_df = pd.DataFrame(numeric_value_poly, columns=poly.get_feature_names_out(columns_to_convert))  # Tambahkan ini
    numeric_value_scaled = scaler.transform(numeric_value_poly_df)  # Ubah ini
    energi_scaled = model.predict(numeric_value_scaled)
    energi = target_scaler.inverse_transform(energi_scaled.reshape(-1, 1))

    predicted_nutrition = pd.DataFrame({
        'ENERGI': np.round(energi.flatten() * (portion_size / 100), 2),
        'PROTEIN': np.round(searched_data['PROTEIN'].values * (portion_size / 100), 2),
        'LEMAK': np.round(searched_data['LEMAK'].values * (portion_size / 100), 2)
    })

    return predicted_nutrition

# Result or Model Testing

In [ ]:
nama_makanan = input("Masukkan nama makanan: ")
portion_size = float(input("Enter portion size (in grams): "))

try:
    result = predictor(nama_makanan, dataset_cleaned, scaler, best_rf_model, target_scaler, poly, portion_size)
    print(f"Predicted nutritional values for '{nama_makanan}' (portion size: {portion_size}g):")
    print(result[['ENERGI', 'PROTEIN', 'LEMAK']])
except ValueError as e:
    print(e)

Masukkan nama makanan: mi basah
Enter portion size (in grams): 100
Predicted nutritional values for 'mi basah' (portion size: 100.0g):
   ENERGI  PROTEIN  LEMAK
0  145.62      6.0   33.0


In [ ]:
import joblib
import json

# Save model
model_filename = 'best_rf_model.pkl'
joblib.dump(best_rf_model, model_filename)

# Save JSON file
model_architecture = best_rf_model.get_params()
model_architecture_filename = 'best_rf_model_architecture.json'
with open(model_architecture_filename, 'w') as json_file:
    json.dump(model_architecture, json_file)

print(f"Model saved to {model_filename} and architecture to {model_architecture_filename}")

Model saved to best_rf_model.pkl and architecture to best_rf_model_architecture.json


# Cek MAE & MSE

In [ ]:
# Import library yang diperlukan
from sklearn.metrics import mean_absolute_error

# Prediksi menggunakan model yang telah dilatih
y_test_pred = best_rf_model.predict(X_test)

# Hitung Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_test_pred)
print("Mean Absolute Error (MAE):", mae)


Mean Absolute Error (MAE): 0.03702824951074751


In [ ]:
# Import library yang diperlukan
from sklearn.metrics import mean_squared_error

# Prediksi menggunakan model yang telah dilatih
y_test_pred = best_rf_model.predict(X_test)

# Hitung Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_test_pred)
print("Mean Squared Error (MSE):", mse)


Mean Squared Error (MSE): 0.004358571360757747


# Versi Python

In [ ]:
import sys
print(sys.version)


3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
